# Metadata

```
course:   DS 5001
module:   00 Final Projects
topic:    Using SpaCy 
author:   R.C. Alvarado
```

# Notes

## How to install

* `conda install -c conda-forge spacy`
* `python -m spacy download en_core_web_sm`

## About SpaCy

<img src="https://spacy.io/architecture-415624fc7d149ec03f2736c4aa8b8f3c.svg" width="500" />

# Set Up

## Config

In [1]:
data_home = "../data"
local_lib = "../lib"
data_prefix = 'novels'
OHCO = ['book_id','chap_id','para_num','sent_num','token_num']

## Import Library

In [23]:
import pandas as pd
import numpy as np
import tqdm as tqdm
import spacy

# Load English resource package

**Trained pipeline packages for English:**
* `en_core_web_sm`: English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer.
* `en_core_web_md`:  English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer.
* `en_core_web_lg`:  English pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, ner, attribute_ruler, lemmatizer.
* `en_core_web_trf`: English transformer pipeline (roberta-base). Components: transformer, tagger, parser, ner, attribute_ruler, lemmatizer.

In [3]:
lang_mod = 'en_core_web_sm'

In [4]:
nlp = spacy.load(lang_mod)

# Import CORPUS

In [5]:
LIB = pd.read_csv(f"{data_home}/{data_prefix}/{data_prefix}-LIB.csv").set_index(OHCO[:1])

In [6]:
CORPUS = pd.read_csv(f"{data_home}/{data_prefix}/{data_prefix}-CORPUS.csv").set_index(OHCO)

In [7]:
CORPUS.head()

pos     term_str
book_id         chap_id para_num sent_num token_num                  
secretadversary 1       0        1        0           DT          the
                                          1          NNP        young
                                          2          NNP  adventurers
                                          3          NNP          ltd
                        1        0        0           JJ        tommy

In [8]:
def gather_docs(CORPUS, ohco_level, str_col='term_str', glue=' '):
    OHCO = CORPUS.index.names
    CORPUS[str_col] = CORPUS[str_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[str_col].apply(lambda x: glue.join(x)).to_frame('doc_str')
    return DOC

In [9]:
SENTS = gather_docs(CORPUS, 4)
CHAPS = gather_docs(SENTS, 2, str_col='doc_str', glue='. ')

In [10]:
CHAPS

doc_str
book_id    chap_id                                                   
adventures 1        a scandal in bohemia. i. to sherlock holmes sh...
           2        the red headed league. i had called upon my fr...
           3        a case of identity. my dear fellow said sherlo...
           4        the boscombe valley mystery. we were seated at...
           5        the five orange pips. when i glance over my no...
...                                                               ...
udolpho    54       vi. unnatural deeds do breed unnatural trouble...
           55       vii. but in these cases we still have judgment...
           56       viii. then fresh tears stood on her cheek as d...
           57       ix. now my task is smoothly done i can fly or ...
usher      1        fall of the house of usher. son coeur est un l...

[320 rows x 1 columns]

# Use SpaCy

In [68]:
# disable=["tok2vec", "tagger", "parser", "ner", "attribute_ruler", "lemmatizer"]
disable=["attribute_ruler", "lemmatizer", "tok2vec", "parser"]
M = nlp.pipe(CHAPS.doc_str.values, disable=disable)
DOCS = [doc.to_json() for doc in M]

In [69]:
DOCS[0].keys()

dict_keys(['text', 'ents', 'sents', 'tokens'])

In [79]:
# DOCS[0]['text']

In [86]:
pd.DataFrame(DOCS[0]['ents']).set_index(['start','end'])

,,label
start,end,
22,24,ORG
244,255,PERSON
279,282,CARDINAL
663,675,PERSON
890,894,PERSON
...,...,...
43254,43259,CARDINAL
44339,44358,TIME
44550,44572,GPE


In [84]:
pd.DataFrame(DOCS[0]['sents']).set_index(['start','end'])

,
start,end
0,21
22,43
44,68
69,126
127,190
...,...
44360,44495
44496,44646
44647,44697


In [85]:
pd.DataFrame(DOCS[0]['tokens']).set_index(['start','end'])

,,id,pos,tag,dep,head
start,end,,,,,
0,1,0,DET,DT,det,1
2,9,1,NOUN,NN,ROOT,1
10,12,2,ADP,IN,prep,1
13,20,3,NOUN,NN,pobj,2
20,21,4,PUNCT,.,punct,1
...,...,...,...,...,...,...
44831,44841,9291,ADJ,JJ,amod,9292
44842,44847,9292,NOUN,NN,pobj,9289
44848,44850,9293,ADP,IN,prep,9292


In [65]:
span = DOCS[0]['tokens'][6]['start'], DOCS[0]['tokens'][6]['end']

In [66]:
DOCS[0]['text'][span[0]:span[1]]

'to'

In [88]:
# DOCS[0][0]

In [61]:
nlp.pipe?

Signature:
nlp.pipe(
    texts,
    as_tuples=False,
    n_threads=-1,
    batch_size=1000,
    disable=[],
    cleanup=False,
    component_cfg=None,
    n_process=1,
)
Docstring:
Process texts as a stream, and yield `Doc` objects in order.

texts (iterator): A sequence of texts to process.
as_tuples (bool): If set to True, inputs should be a sequence of
    (text, context) tuples. Output will then be a sequence of
    (doc, context) tuples. Defaults to False.
batch_size (int): The number of texts to buffer.
disable (list): Names of the pipeline components to disable.
cleanup (bool): If True, unneeded strings are freed to control memory
    use. Experimental.
component_cfg (dict): An optional dictionary with extra keyword
    arguments for specific components.
n_process (int): Number of processors to process texts, only supported
    in Python3. If -1, set `multiprocessing.cpu_count()`.
YIELDS (Doc): Documents in the order of the original text.

DOCS: https://spacy.io/api/language#pip